In [7]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Store csv created in part one into a DataFrame
city_data = pd.read_csv("output_data/cities.csv")
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,voloshka,61.3314,40.0858,31.93,97,100,1.43,RU,1616777715
1,1,rikitea,-23.1203,-134.9692,79.25,71,9,20.18,PF,1616777716
2,2,popondetta,-8.7537,148.2534,74.01,90,100,2.89,PG,1616777716
3,3,laguna,38.4210,-121.4238,57.00,47,1,12.66,US,1616777716
4,4,stutterheim,-32.5708,27.4240,71.60,78,0,12.66,ZA,1616777717


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)
ciy_data = city_data.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)

hum = city_data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = hum, 
                                 dissipating=False, max_intensity = 100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

In [26]:
# Heatmap of humidity
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data.loc[(city_data["Max Temp"] >= 60) & (city_data["Max Temp"] <= 75) & (city_data["Humidity"] < 50) 
                        & (city_data["Cloudiness"] < 20) & (city_data["Wind Speed"] < 10)]
narrow_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,san carlos de bariloche,-41.1456,-71.3082,73.00,34,0,4.61,AR,1616777503
78,78,medea,36.2642,2.7539,71.60,37,0,6.91,DZ,1616777737
80,80,galeana,24.8333,-100.0667,69.01,18,0,1.99,MX,1616777738
114,114,lasa,46.6166,10.7002,66.20,46,0,8.05,IT,1616777747
121,121,dharchula,29.8500,80.5333,60.28,24,0,7.63,IN,1616777748
302,302,buin,-33.7333,-70.7500,66.99,42,0,6.91,CL,1616777667
319,319,pokhara,28.2333,83.9833,66.94,17,0,7.14,NP,1616777804
325,325,sulgen,47.5377,9.1850,60.80,47,0,4.61,CH,1616777805
355,355,myitkyina,25.3833,97.4000,69.03,31,15,1.88,MM,1616777814
392,392,zhongshu,24.5167,103.7667,64.11,35,1,9.33,CN,1616777825


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = pd.DataFrame(narrow_df[["City", "Country", "Lat", "Lng"]])
hotel_df["Hotel Name"] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
41,san carlos de bariloche,AR,-41.1456,-71.3082,
78,medea,DZ,36.2642,2.7539,
80,galeana,MX,24.8333,-100.0667,
114,lasa,IT,46.6166,10.7002,
121,dharchula,IN,29.8500,80.5333,
302,buin,CL,-33.7333,-70.7500,
319,pokhara,NP,28.2333,83.9833,
325,sulgen,CH,47.5377,9.1850,
355,myitkyina,MM,25.3833,97.4000,
392,zhongshu,CN,24.5167,103.7667,


In [56]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    data = response.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
41,san carlos de bariloche,AR,-41.1456,-71.3082,Hampton by Hilton Bariloche
78,medea,DZ,36.2642,2.7539,Hôtel M'Sallah
80,galeana,MX,24.8333,-100.0667,Hotel Cuatro reales
114,lasa,IT,46.6166,10.7002,Gasthof Hotel Sonne Silandro
121,dharchula,IN,29.8500,80.5333,Hotel new yash dharchula
302,buin,CL,-33.7333,-70.7500,Paihuen
319,pokhara,NP,28.2333,83.9833,Shangrila Village Resort Pokhara
325,sulgen,CH,47.5377,9.1850,Hotel Le Lion
355,myitkyina,MM,25.3833,97.4000,Cartel Hotel
392,zhongshu,CN,24.5167,103.7667,Fuding Hotel


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# Add marker layer ontop of heat map

# Display figure
